# Solutions
## Hartree-Fock
## Configuration interaction
### Several states

In [55]:
import veloxchem as vlx
import multipsi as mtp
import numpy as np
O2_xyz="""2
O2                                                                                                                         
O    0.000000000000        0.000000000000       -0.600000000000 
O    0.000000000000        0.000000000000        0.600000000000 
"""

molecule = vlx.Molecule.from_xyz_string(O2_xyz)
molecule.set_multiplicity(3)
basis = vlx.MolecularBasis.read(molecule,"STO-3G")

scfdrv = vlx.ScfUnrestrictedDriver()
scfdrv.compute(molecule, basis)

space=mtp.OrbSpace(molecule,scfdrv.mol_orbs)
space.FCI(nfrozen=4)

expansion=mtp.CIExpansion(space)
CIham=mtp.CIOperator(expansion) #Contains integrals and functions using them
CIham.compute_Hints(molecule,basis)

Ein= float(CIham.inEne)
Ftu= CIham.Ftu
tuvw= CIham.tuvw

def SC_diag(occa, occb):
    '''
    The energy of a given SD, as a function of its list of occupied orbitals
    '''
    Hij=Ein #Inactive energy (inc. nuclear repulsion)
    for i in occa:
        Hij+=Ftu[i,i] #1-e term = inactive Fock matrix
        for j in occa:
            if i<j:
                Hij+=tuvw[i,i,j,j]-tuvw[i,j,j,i] #Coulomb-Exchange
        for j in occb:
            Hij+=tuvw[i,i,j,j]
    for i in occb:
        Hij+=Ftu[i,i]
        for j in occb:
            if i<j:
                Hij+=tuvw[i,i,j,j]-tuvw[i,j,j,i]
    return Hij
def SC_1exc(i,a,ss_occ, os_occ):
    '''
    Slater-Condon between a SD and a singly excited, depending on the excited orbitals (i,a)
    and the same-spin (compared to spin of the excitated electron) and opposite-spin occupation
    '''
    Hij=Ftu[i,a]
    for k in ss_occ:
        Hij+=tuvw[i,a,k,k]-tuvw[i,k,k,a]
    for k in os_occ:
        Hij+=tuvw[i,a,k,k]
    return Hij
def SC_ss1exc(i,a,j,b):
    '''
    Slater-Condon between a SD and a doubly excited determinant,
    with both excited electrons having the same spin
    '''
    return tuvw[i,a,j,b]-tuvw[i,b,j,a]
def SC_os1exc(i,a,j,b):
    '''
    Slater-Condon between a SD and a doubly excited determinant,
    with the excited electrons having opposite spin
    '''
    return tuvw[i,a,j,b]

def sigma(vector):
    result=np.zeros(expansion.nDet)
    for idet,det in enumerate(expansion.detlist()):
        #Diagonal term
        result[idet]+=SC_diag(det.occ_alpha(),det.occ_beta())*vector[idet]
        #Single excitations alpha
        for i in det.occ_alpha():
            for a in det.unocc_alpha():
                phase,excdet=det.excite_alpha(i,a)
                result[excdet.index()]+=phase*SC_1exc(i,a,det.occ_alpha(),det.occ_beta())*vector[idet]
                #alpha-alpha excitation
                for j in det.occ_alpha():
                    if i>=j:
                        continue
                    for b in det.unocc_alpha():
                        if a>=b:
                            continue
                        phase2,exc2det=excdet.excite_alpha(j,b)
                        result[exc2det.index()]+=phase*phase2*SC_ss1exc(i,a,j,b)*vector[idet]
                #alpha-beta excitation
                for j in det.occ_beta():
                    for b in det.unocc_beta():
                        phase2,exc2det=excdet.excite_beta(j,b)
                        result[exc2det.index()]+=phase*phase2*SC_os1exc(i,a,j,b)*vector[idet]
        #Single excitations beta
        for i in det.occ_beta():
            for a in det.unocc_beta():
                phase,excdet=det.excite_beta(i,a)
                result[excdet.index()]+=phase*SC_1exc(i,a,det.occ_beta(),det.occ_alpha())*vector[idet]
                #beta-beta excitation
                for j in det.occ_beta():
                    if i>=j:
                        continue
                    for b in det.unocc_beta():
                        if a>=b:
                            continue
                        phase2,exc2det=excdet.excite_beta(j,b)
                        result[exc2det.index()]+=phase*phase2*SC_ss1exc(i,a,j,b)*vector[idet]
    return result

np.set_printoptions(formatter={'float_kind':"{:.3f}".format})

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Hartree-Fock                                       
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

In [56]:
nstates = 3

#Compute Hdiag
Hdiag=np.empty(expansion.nDet)
for idet,det in enumerate(expansion.detlist()):
    Hdiag[idet]=SC_diag(det.occ_alpha(),det.occ_beta())

idx=np.argsort(Hdiag,nstates)[:nstates]
vecs=[]
for i in range(nstates):
    veci=np.zeros(expansion.nDet)
    veci[idx[i]]=1
    vecs.append(veci)

resnorm=1
istep=0
while resnorm>0.0001: #As long as the residual norm is large
    istep+=1
    
    sigmas=[]
    energies=[]
    for i in range(nstates):
        sigmas.append(sigma(vecs[i]))
        energies.append(np.dot(vecs[i],sigmas[i]))
    print("Energies at step",istep,"=",energies)
    
    #Compute residual and its norm
    residuals=[]
    resnorm=0
    for i in range(nstates):
        residuals.append(sigmas[i]-energies[i]*vecs[i])
        resnorm=max(resnorm,np.linalg.norm(residuals[i]))
    
    #Compute Davidson update
    vec1s=[]
    for i in range(nstates):
        preconditioner=1/(Hdiag-energies[i]+0.0001) #0.0001 to prevent divergence
        vec1s.append(preconditioner*residuals[i])
    
    #Orthonormalize new vectors with old ones and themselves
    sigvec1s=[]
    for i in range(nstates):
        newvec1=np.array(vec1s[i])
        for j in range(nstates):
            newvec1-=np.dot(vec1s[i],vecs[j])*vecs[j]
        for j in range(i):
            newvec1-=np.dot(vec1s[i],vec1s[j])*vec1s[j]
        norm = np.linalg.norm(newvec1)
        vec1s[i]=newvec1/norm
        sigvec1s.append(sigma(vec1s[i]))
    
    #Create small hamiltonian
    smallHam=np.zeros((2*nstates,2*nstates))
    for i in range(nstates):
        for j in range(nstates):
            smallHam[i,j]=np.dot(vecs[i],sigmas[j])
            smallHam[i,j+nstates]=np.dot(vecs[i],sigvec1s[j])
            smallHam[i+nstates,j]=np.dot(vec1s[i],sigmas[j])
            smallHam[i+nstates,j+nstates]=np.dot(vec1s[i],sigvec1s[j])
    
    #Form the updated CI vector using the eigenvector
    w,v=np.linalg.eigh(smallHam)
    newvecs=[]
    for i in range(nstates):
        vec0=np.zeros(expansion.nDet)
        for j in range(nstates):
            vec0+= v[j,i]*vecs[j]
            vec0+= v[j+nstates,i]*vec1s[j]
        norm = np.linalg.norm(vec0)
        newvecs.append(vec0/norm)

    vecs=newvecs        

AxisError: axis 3 is out of bounds for array of dimension 1

### 1-particle density matrix

In [4]:
import veloxchem as vlx
import multipsi as mtp
import numpy as np
O2_xyz="""2
O2                                                                                                                         
O    0.000000000000        0.000000000000       -0.600000000000 
O    0.000000000000        0.000000000000        0.600000000000 
"""

molecule = vlx.Molecule.from_xyz_string(O2_xyz)
basis = vlx.MolecularBasis.read(molecule,"STO-3G")

scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(molecule, basis)

molecule.set_multiplicity(3)

space=mtp.OrbSpace(molecule,scfdrv.mol_orbs)
space.CAS(8,6)
expansion=mtp.CIExpansion(space)

CIdrv=mtp.CIDriver(molecule,basis,space)
CIdrv.compute(1)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 4
Number of active orbitals:              6
Number of virtual orbitals:             0

    This is a CASSCF wavefunction: CAS(8,6)

          CI expansion:
          -------------
Number of determinants:      120



        CI Iterations
        -------------

Iteration   Average energy      Time
     1        -147.72142572   0:00:00
Convergence reached in 1 iterations

        Final results
        -------------

* State 1
- Energy: -147.7214257188092
- S^2   : 2.00  (multiplicity = 3.0 )
- Natural orbitals
[1.96865 1.95514 1.95514 1.04412 1.04412 0.03284]


Now let's implement the density matrix:

In [5]:
def Den1(vector):
    DM=np.zeros((space.nAct,space.nAct))
    
    for idet,det in enumerate(expansion.detlist()):
        #Diagonal term
        result=vector[idet]*vector[idet]
        for i in det.occ_alpha():
            DM[i,i]+= result
        for i in det.occ_beta():
            DM[i,i]+= result
        #Single excitations alpha
        for i in det.occ_alpha():
            for a in det.unocc_alpha():
                phase,excdet=det.excite_alpha(i,a)
                DM[a,i]+= phase* vector[idet]* vector[excdet.index()]
        #Single excitations beta
        for i in det.occ_beta():
            for a in det.unocc_beta():
                phase,excdet=det.excite_beta(i,a)
                DM[a,i]+= phase*vector[idet] * vector[excdet.index()]
    return DM

In [6]:
#Check that the matrices match
vec0=CIdrv.vecs[0].to_numpy()
Dpq=Den1(vec0)
np.testing.assert_almost_equal(Dpq, CIdrv.get1den(0))